In [1]:
import sys
sys.path

['',
 '/usr/local/lib/python3.5/dist-packages',
 '/home/kira/catkin_ws/devel/lib/python2.7/dist-packages',
 '/opt/ros/kinetic/lib/python2.7/dist-packages',
 '/usr/local/share/opencog/python',
 '/home/kira/opencog/build/opencog/python',
 '/home/kira/opencog/build/opencog/cython',
 '/home/kira/opencog/opencog/python',
 '/home/kira/anaconda3/lib/python36.zip',
 '/home/kira/anaconda3/lib/python3.6',
 '/home/kira/anaconda3/lib/python3.6/lib-dynload',
 '/home/kira/anaconda3/lib/python3.6/site-packages',
 '/home/kira/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/kira/.ipython']

# Making some Atoms in python

Here we will mostly use the ipython shell to give examples. But you can of course put all the commands in a *script_name.py* file if you like and execute it, on terminal, using the command :

*$ python script_name.py* 

In [2]:
# Import AtomSpace and types to start making atoms.
from opencog.atomspace import AtomSpace, types

/home/kira/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'opencog.atomspace' does not match runtime version 3.6
  return f(*args, **kwds)


There are two ways to create Atoms in python. You can use the c++ like syntax or you can use the scheme like syntax.

# c++ like syntax

In [3]:
#declare an atomspace called atsp
atsp = AtomSpace()

In [4]:
#initializing atoms(nodes) of type ConceptNode.
cat = atsp.add_node(types.ConceptNode, "Cat")
animal = atsp.add_node(types.ConceptNode, "Animal")
man = atsp.add_node(types.ConceptNode, "Man")

In [5]:
#print a node
man

(ConceptNode "Man") ; [4943478229863008194][1]

Printing the AtomSpace will not quite work out as we might expect from c++. We can print its contents as follows:

In [6]:
#print the atomsapce
for atom in atsp:
    print (atom)

(ConceptNode "Cat")

(ConceptNode "Man")

(ConceptNode "Animal")



Now lets add some links into the AtomSpace. This is again, just like c++.

In [7]:
#initialize atom of type InheritanceLink.
atsp.add_link(types.InheritanceLink, [man, animal])

(InheritanceLink
  (ConceptNode "Man") ; [4943478229863008194][1]
  (ConceptNode "Animal") ; [1051227453015520217][1]
) ; [17186793166877131821][1]

# scheme like syntax

To use scheme like concise syntax you have to do a few extra steps. Import the *type_constructors* that enable you to add atoms to the AtomSpace by calling functions named after their types. Import *initialize_opencog*, a function that binds an AtomSpace instance (*atsp* in our case) to the *type_constructors*.

In [8]:
#Import initialize_opencog
from opencog.utilities import initialize_opencog

/home/kira/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'opencog.utilities' does not match runtime version 3.6
  return f(*args, **kwds)
/home/kira/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'opencog.type_constructors' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
#Import type_constructors
from opencog.type_constructors import *

Now you can add atoms just like in scheme. This becomes very useful when you start building complex patterns for the *pattern matcher*.

In [10]:
#bind atsp with the type_constructors
initialize_opencog(atsp)

In [11]:
#initialize and add more atoms into atsp
color = ConceptNode("Color")
c1 = ConceptNode("Blue")
link = InheritanceLink(c1, color)

In [12]:
#print the atomspace
for atom in atsp:
    print (atom)

(ConceptNode "Blue")

(ConceptNode "Cat")

(ConceptNode "Color")

(ConceptNode "Man")

(ConceptNode "Animal")

(InheritanceLink
  (ConceptNode "Blue")
  (ConceptNode "Color")
)

(InheritanceLink
  (ConceptNode "Man")
  (ConceptNode "Animal")
)



One last thing before we wrap this up. The Atoms are classes with a lot of useful methods. If you are in the ipython shell you can look at a list of the methods by writing *< variable_pointing_to_an_atom >* . *< Tab >*. For example, we have a variable called *"color"* that points to the *ConceptNode* object of name *"Color"*. Then we can see a list of all the methods of *"color"* by :

*color.< Press Tab >*

# Pattern Matching

Now lets look at how we can run the PM through python. The PM functions are defined in c++ and made available in python through wrappers that are defined in atomspace/opencog/cython/opencog/bindlink.pyx and you can see the imports in *atomspace/opencog/cython/opencog/bindlink.pxd*

Here we will again run the equivalent of *bindlink* and *cog-staisfying-set* functions in scheme.

In [13]:
#Import the satisfying_set function
from opencog.bindlink import satisfying_set

/home/kira/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'opencog.bindlink' does not match runtime version 3.6
  return f(*args, **kwds)


In [14]:
#Delete existing atoms
atsp.clear()
#NOTE that the atoms are deleted only from atsp not from the entire python environment. 

In [15]:
#Create some atoms
InheritanceLink(ConceptNode("Red"), color)
InheritanceLink(ConceptNode("Green"), color)
InheritanceLink(ConceptNode("Blue"), color)

(InheritanceLink
  (ConceptNode "Blue") ; [3109859930626802082][1]
  (ConceptNode "Color") ; [8429230717376582988][1]
) ; [10172252740717271936][1]

In [16]:
#Create a pattern to look for color nodes
varlink = TypedVariableLink(VariableNode("$xcol"), TypeNode("ConceptNode"))
pattern = InheritanceLink(VariableNode("$xcol"), color)
colornodes = GetLink(varlink, pattern)

In [17]:
#Now run the function to find all color nodes
satisfying_set(atsp, colornodes)

(SetLink
  (ConceptNode "Blue") ; [3109859930626802082][1]
  (ConceptNode "Red") ; [6709574915108404932][1]
  (ConceptNode "Green") ; [7362795004147159697][1]
) ; [16291863344720241133][1]

In a similar manner you can import *bindlink* from *opencog.bindlink* and use it to execute rewrite queries (with the help of *BindLink* in place of *SatisfactionLink*)